# 서울시 기초생활수급자 전처리

In [1]:
import pandas as pd
import numpy as np

In [3]:
#데이터 불러들이기
basic_df = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/서울특별시 국민기초생활 수급자 동별 현황_20210731_v1.csv',
                      encoding = 'cp949')

In [19]:
basic_df = basic_df[basic_df['연령구간'] == '65세이상']
basic_df = basic_df[(basic_df['자격'] == '기초생계급여') | (basic_df['자격'] == '기초의료급여')]
basic_df.reset_index(drop = True, inplace = True)

In [22]:
#동이 '구' 로 끝나는 데이터 삭제
for i in range(len(basic_df)):
    if basic_df['읍면동'][i][-1] == '구':
        basic_df.drop(i, axis = 0, inplace = True)

basic_df.reset_index(drop = True, inplace = True)
    

In [26]:
#group_by
basic_df1 = basic_df.groupby(['시군구', '읍면동']).sum()

In [33]:
basic_df1.rename(columns = {'수급권자수' : '국민기초생활 노인 인구'}, inplace = True)


In [35]:
basic_df1.to_csv('C:/Users/psj70/Downloads/DNA Data Archive/서울특별시 국민기초생활 노인 인구.csv')

# 생활인구 데이터 전처리 (2022.06 ~ 2022.10 일별노인생활인구 평균)

In [305]:
import pandas as pd
import numpy as np

In [306]:
#데이터 불러오기
df6 = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/동별 생활인구_202206.csv')
df7 = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/동별 생활인구_202207.csv')
df8 = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/동별 생활인구_202208.csv')
df9 = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/동별 생활인구_202209.csv')
df10 = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/동별 생활인구_202210.csv')


In [307]:
#인덱스 밀린 것 해결
for i in [df6, df7, df8, df9, df10]:
    columns_list = i.columns
    i.insert(0, 'new', i.index)
    i.drop(['여자70세이상생활인구수'], axis =1, inplace = True)
    i.columns = columns_list
    i.reset_index(drop= True, inplace = True)
    i = i[['남자65세부터69세생활인구수', '남자70세이상생활인구수','여자65세부터69세생활인구수', '여자70세이상생활인구수']]


In [308]:
#데이터 결합
total_df = pd.concat([df6, df7, df8, df9, df10], axis = 0)
total_df.reset_index(drop = True, inplace = True)

In [281]:
#필요한 컬럼들만
total_df = total_df[['행정동코드', '남자65세부터69세생활인구수', '남자70세이상생활인구수','여자65세부터69세생활인구수', '여자70세이상생활인구수']]

In [282]:
total_df['노인인구'] = total_df['남자65세부터69세생활인구수'] + total_df['남자70세이상생활인구수'] + total_df['여자65세부터69세생활인구수']+ total_df['여자70세이상생활인구수']
total_df = total_df[['행정동코드', '노인인구']]

In [283]:
code_df = total_df.groupby('행정동코드').mean()

In [284]:
code_df['행정동 코드'] = code_df.index
code_df.reset_index(drop = True, inplace = True)

In [285]:
code_df

,노인인구,행정동 코드
0,2870.564993,11110515
1,3177.154042,11110530
2,742.118114,11110540
3,2697.490635,11110550
4,3704.634670,11110560
...,...,...
419,4334.506353,11740650
420,4328.904847,11740660
421,10447.084832,11740685
422,781.306647,11740690


## 행정동 코드 뽑기

In [286]:
df = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/서울시 행정동코드.csv', encoding = 'cp949')


In [287]:
df = df[['행정구역명', '행정기관코드']]

In [288]:
df.drop_duplicates(keep = 'last', inplace = True)
df.reset_index(drop = True, inplace=True)

In [289]:
for i in range(len(df)):
    df['행정기관코드'][i] = int(str(df['행정기관코드'][i])[:-2])

<ipython-input-289-23d135cf8751>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['행정기관코드'][i] = int(str(df['행정기관코드'][i])[:-2])


In [290]:
code_df1 = pd.merge(code_df, df, left_on = '행정동 코드', right_on = '행정기관코드', how = 'left')

In [291]:
code_df1

,노인인구,행정동 코드,행정구역명,행정기관코드
0,2870.564993,11110515,청운효자동,11110515.0
1,3177.154042,11110530,사직동,11110530.0
2,742.118114,11110540,삼청동,11110540.0
3,2697.490635,11110550,부암동,11110550.0
4,3704.634670,11110560,평창동,11110560.0
...,...,...,...,...
419,4334.506353,11740650,성내2동,11740650.0
420,4328.904847,11740660,성내3동,11740660.0
421,10447.084832,11740685,길동,11740685.0
422,781.306647,11740690,둔촌1동,11740690.0


## 결측치 채우기

In [292]:
#대중교통 동 목록
aa = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/1125 서울 동 별 대중교통 개수 정보.csv')

In [293]:
b = list(aa['동'])

In [294]:
#없는 동 리스트
for i in range(len(b)):
    if b[i] not in list(code_df1['행정구역명']):
        print(b[i])

번1동
번2동
번3동
수유1동
수유2동
수유3동
항동
상일1동
상일2동


In [295]:
code_df1.loc[135, '행정구역명'] = '번1동'
code_df1.loc[136, '행정구역명'] = '번2동'
code_df1.loc[137, '행정구역명'] = '번3동'
code_df1.loc[138, '행정구역명'] = '수유1동'
code_df1.loc[139, '행정구역명'] = '수유2동'
code_df1.loc[140, '행정구역명'] = '수유3동'
code_df1.loc[135, '행정구역명'] = '번1동'

In [296]:
#필요한 컬럼들만
code_df1 = code_df1[['행정구역명', '노인인구']]

In [298]:
#상일동을 상일1동, 상일2동 노인인구 비율 (고령자현황 데이터)로 나누어줌
code_df1.loc[424] = ['상일1동', 6140.69 * (4730/(4730 + 1888))]
code_df1.loc[425] = ['상일2동', 6140.69 * (1888/(4730 + 1888))]

C:\Users\psj70\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [299]:
code_df1.dropna(inplace = True)
code_df1.reset_index(drop = True, inplace = True)

<ipython-input-299-1ac4c8b6b68e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_df1.dropna(inplace = True)


In [301]:
#항동은 오류2동으로부터 분리되어 있으므로 노인인구 비율에 따라 오류2동, 항동 노인 수 조정
#오류2동 
code_df1.loc[273, '노인인구'] = 7948.75 * (7361 / (7361 + 1762))
#항동
code_df1.loc[426] = ['항동', 7948.75 * (1762 / (7361 + 1762))]

C:\Users\psj70\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\psj70\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [302]:
code_df1['노인인구'] = np.round(code_df1['노인인구'],0)

<ipython-input-302-2656cce9c7cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_df1['노인인구'] = np.round(code_df1['노인인구'],0)


In [303]:
code_df1.columns = ['행정동명', '일별 노인생활인구 평균']

In [304]:
code_df1.to_csv('C:/Users/psj70/Downloads/DNA Data Archive/서울시 일별 노인생활인구 평균.csv')

# 교통관련 변수 정리

In [8]:
import pandas as pd
import numpy as np

In [9]:
transport_df = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/1125 서울 동 별 대중교통 개수 정보.csv')
pop_df = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/서울시 동별 고령자.csv')

In [10]:
#필요한 컬럼들만
transport_df = transport_df[['구', '동', '지하철 개수 보정', '정류장 개수', '면적 (k㎡)', '면적(k㎡) 대비 정류장개수 비율']]
pop_df = pop_df[['동별(2)', '동별(3)', '전체인구', '65세이상 인구']]

In [11]:
#상일2동 값 수정
pop_df.loc[450, '65세이상 인구'] = 1492

In [12]:
#소계 삭제
pop_df = pop_df[pop_df['동별(3)'] != '소계'] 

In [13]:
#관악구, 강남구 신사동 재정의 
pop_df.loc[350,'동별(3)' ] = '신사동'
pop_df.loc[381,'동별(3)' ] = '신사동'

In [14]:
# 인구, 교통 데이터프레임 결합
final_df = pd.merge(transport_df, pop_df, left_on = ['구', '동'], right_on = ['동별(2)', '동별(3)'], how = 'outer')

In [15]:
#변수 생성 및 필요없는 컬럼 제거
final_df.drop(['동별(2)', '동별(3)', '면적 (k㎡)'], axis = 1 , inplace = True)
final_df['인구 대비 정류장 개수'] = final_df['정류장 개수'] / final_df['전체인구']
final_df['인구 대비 지하철 개수'] = final_df['지하철 개수 보정'] / final_df['전체인구']

# 인구관련 변수 정리

In [16]:
#데이터 불러오기
living_df = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/서울시 일별 노인생활인구 평균.csv', encoding = 'cp949')
basic_df = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/서울특별시 국민기초생활 노인 인구.csv', encoding = 'cp949')

In [17]:
#65세 이상 인구비율
final_df['65세이상 인구 비율'] = final_df['65세이상 인구'] / final_df['전체인구']

In [18]:
#pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [19]:
final_df = pd.merge(final_df, basic_df, left_on = ['구', '동'], right_on = ['시군구', '읍면동'], how = 'outer')

In [20]:
final_df = pd.merge(final_df, living_df, left_on = ['구', '동'], right_on = ['Unnamed: 0', '행정동명'], how = 'outer')

In [21]:
#파생변수 생성
final_df['국민기초생활 노인 비율'] = final_df['국민기초생활 노인 인구']/ final_df['65세이상 인구']


In [22]:
#컬럼 제거
final_df.drop(['전체인구','시군구', '읍면동', 'Unnamed: 0', '행정동명'], axis = 1, inplace = True)

In [23]:
final_df

,구,동,지하철 개수 보정,정류장 개수,면적(k㎡) 대비 정류장개수 비율,65세이상 인구,인구 대비 정류장 개수,인구 대비 지하철 개수,65세이상 인구 비율,국민기초생활 노인 인구,일별 노인생활인구 평균,국민기초생활 노인 비율
0,종로구,사직동,2.0,34.0,27.642276,1774,0.003528,0.000208,0.184101,65,3177,0.036640
1,종로구,삼청동,0.0,18.0,12.080537,625,0.006572,0.000000,0.228185,19,742,0.030400
2,종로구,부암동,0.0,35.0,15.418502,1802,0.003578,0.000000,0.184216,61,2697,0.033851
3,종로구,평창동,0.0,53.0,5.975197,3435,0.002892,0.000000,0.187408,48,3705,0.013974
4,종로구,무악동,0.0,5.0,13.888889,1424,0.000603,0.000000,0.171628,89,1334,0.062500
5,종로구,교남동,0.0,15.0,42.857143,1624,0.001470,0.000000,0.159169,78,1144,0.048030
6,종로구,가회동,0.0,21.0,38.888889,865,0.005007,0.000000,0.206247,27,929,0.031214
7,종로구,종로1.2.3.4가동,5.0,72.0,30.638298,1960,0.008377,0.000582,0.228040,336,8302,0.171429
8,종로구,종로5.6가동,2.0,16.0,26.666667,1057,0.002922,0.000365,0.193024,110,3102,0.104068
9,종로구,이화동,1.0,21.0,26.923077,1360,0.002726,0.000130,0.176509,96,3126,0.070588


# 등고선 데이터 정리

In [24]:
#데이터 불러오기
steep_df = pd.read_csv('C:/Users/psj70/Downloads/DNA Data Archive/서울시 행정구역별 평균 경사도.csv', encoding = 'cp949')

In [25]:
steep_df

,구,행정동,평균 경사도
0,종로구,사직동,5.155626
1,종로구,삼청동,6.947532
2,종로구,부암동,16.601789
3,종로구,평창동,16.844632
4,종로구,무악동,12.888912
5,종로구,교남동,8.775398
6,종로구,가회동,7.782677
7,종로구,종로1.2.3.4가동,3.141511
8,종로구,종로5.6가동,4.512152
9,종로구,이화동,7.661186


In [26]:
final_df = pd.merge(final_df, steep_df, left_on = ['구', '동'], right_on = ['구', '행정동'], how = 'outer')

In [27]:
final_df.drop(['행정동'], axis = 1, inplace = True)

# 표준화

In [28]:

label_df = final_df[['구', '동']]
final_df1 = final_df.drop(['구', '동'], axis = 1)

In [29]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
final_df2 = sc.fit_transform(final_df1)
final_df2 = pd.DataFrame(final_df2, columns = list(final_df1.columns))


In [30]:
final_df2 = pd.concat([label_df, final_df2], axis = 1)

In [31]:
final_df2

,구,동,지하철 개수 보정,정류장 개수,면적(k㎡) 대비 정류장개수 비율,65세이상 인구,인구 대비 정류장 개수,인구 대비 지하철 개수,65세이상 인구 비율,국민기초생활 노인 인구,일별 노인생활인구 평균,국민기초생활 노인 비율,평균 경사도
0,종로구,사직동,0.698532,0.523366,0.275995,-1.309322,1.373327,0.354864,0.439349,-0.804374,-0.279992,-0.635534,-0.183835
1,종로구,삼청동,-0.884312,-0.543399,-0.957933,-2.063575,3.272466,-0.222892,1.691459,-0.994676,-1.872351,-0.770168,0.323557
2,종로구,부암동,-0.884312,0.590039,-0.693258,-1.290942,1.404257,-0.222892,0.442604,-0.820922,-0.593886,-0.695708,3.057235
3,종로구,평창동,-0.884312,1.790149,-1.442041,-0.218971,0.975913,-0.222892,0.533266,-0.874703,0.065292,-1.124559,3.125998
4,종로구,무악동,-0.884312,-1.410145,-0.814545,-1.539077,-0.452489,-0.222892,0.085082,-0.705086,-1.485215,-0.077620,2.005905
5,종로구,교남동,-0.884312,-0.743417,1.482418,-1.407789,0.088882,-0.222892,-0.268799,-0.750593,-1.609464,-0.389816,0.841131
6,종로구,가회동,-0.884312,-0.343381,1.167766,-1.906029,2.296103,-0.222892,1.068348,-0.961580,-1.750063,-0.752609,0.560035
7,종로구,종로1.2.3.4가동,3.072800,3.056932,0.513556,-1.187224,4.398993,1.396437,1.687315,0.316752,3.071482,2.272484,-0.754148
8,종로구,종로5.6가동,0.698532,-0.676745,0.198636,-1.779992,0.994789,0.793772,0.692781,-0.618209,-0.329038,0.819201,-0.366040
9,종로구,이화동,-0.092890,-0.343381,0.218967,-1.581089,0.872266,0.138384,0.223700,-0.676127,-0.313343,0.096882,0.525634


In [33]:
final_df.to_csv('C:/Users/psj70/Downloads/DNA Data Archive/클러스터링 데이터프레임(표준화 미시행).csv')

In [34]:
final_df2.to_csv('C:/Users/psj70/Downloads/DNA Data Archive/클러스터링 데이터프레임(표준화 시행).csv')